![alt text](./images/TrainingANN.png "Image produced by callback")


## Keras Training Visualization

This notebook uses a Keras callback and Matplotlib to display an animated graph of a model being trained.<br />

The model trained is a multi-layer neural network. The callback function in the cell titled "Function that draws and updates the graph" is generic and can be used for any neural network. The key limitations are that it's only for a single input and single output.<br />

The basic strategy is to create a hook after each mini-batch training. The callback runs the model on the dataset and plots the results and (if desired) the current mean squared error.<br />

### Import Requirements
Note that this uses qt5 for display, not inline. It is possible to do animation inline, but it's a bit more limiting.

In [1]:
import numpy as np
import pandas as pd
%matplotlib qt5
import matplotlib.pyplot as plt

### Create Data

This is a toy dataset made to show how a neural network can fit complex functions.

In [2]:
# evenly spaced input values
x = np.linspace(0, 12, 10000).reshape(-1,1)

# function that maps features to labels
def f(x):
    # zero, then sin for a bit, then zero again
    if x < np.pi:
        return 0
    elif x < 3 * np.pi:
        return np.sin(x)
    else:
        return 0

# map the labels onto the features
y = np.array([f(v) for v in x]).reshape(-1)

# create labels that are normally distributed around the curve
noise = np.random.randn(len(y)) * 0.5

features = x
labels = y + noise

## Take a quick look at the function and dataset created

In [3]:
plt.figure(figsize=(15, 10))
plt.ylim(-2.5, 2.5)
plt.plot(x, y, lw=3)
plt.scatter(features, labels, s=1, alpha=0.5, c='gray')

### Split into test and training datasets

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features.reshape(-1, 1), labels, random_state=42)

## Import function that draws and updates the graph

See kerviz.py for the code.

The callback function should be run *on_batch_end*. It determines whether an update is necessary and redraws the graph, displaying related data, as well.<br />

The enclosing function does some set up and creates a closure with the data that will be retained between calls or that needs to be known in advance, because keras passes very little information to the callback.<br />

This function should be reusable for graphing any single-input single-output model in Keras. I have used it, e.g., for multi-layer neural networks.<br />

Expect this to be *super slow*. Especially for simple models, the cost of running and graphing the model will be significantly higher than the cost of training on a single batch. You will get warnings from Keras about slowness.<br />

This is a toy for learning / investigating. So, performance is not a primary concern, but you do want it to be usable. Several things can have a big impact on performance.<br />
<ol>
<li>The **sparsity** options reduce the number datapoints used in the scatter plot and in running the model on each pass.</li>
<li>The **frequency** option determines how often to update the graph. I've used the function option to update only when the change to the model parameters is big enough to justify an update. But, it's specific to the model being trained.</li>
<li>Turning off the **loss display** slightly reduces the number of computations, but significantly reduces the amount of drawing.</li>
</ol>

There is also an option to write the updates to files as individual images, which can then be used to create an animation.

In [5]:
from kerviz import get_redraw

## Create and compile the model

In [8]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(200, input_dim=features.shape[1], activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adagrad')

epochs = 75
batch_size = 128


## Build the callback function that will be passed to Keras and train the model

In [9]:
from keras.callbacks import LambdaCallback

# get closured redraw callback function
# this will also draw the background for the graph
cb_redraw = get_redraw( X_train, y_train, model, batch_size, epochs,
                        frequency=20, graph_sparsity=1,
                        scatter_sparsity=1, show_err=True, err_smoothing=201,
                        title="Neural Network Fitting Complex Function",
                        x_label="x",
                        y_label="f(x)",
                        loss_scale=0.8, display_mode='screen')

# wrap callback function in Keras structure, to be called after each batch
redraw_callback = LambdaCallback(on_batch_end=cb_redraw)

# train the model, passing the Keras-wrapped callback function
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[redraw_callback])

Epoch 1/75
7500/7500 [==============================] - 1s 158us/step - loss: 1.2001
Epoch 2/75
 256/7500 [>.............................] - ETA: 8s - loss: 0.4423

/var/root/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.298200). Check your callbacks.
  % delta_t_median)
/var/root/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.150244). Check your callbacks.
  % delta_t_median)


7500/7500 [==============================] - 1s 135us/step - loss: 0.4811
Epoch 3/75
 384/7500 [>.............................] - ETA: 5s - loss: 0.5206

/var/root/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.142806). Check your callbacks.
  % delta_t_median)


7500/7500 [==============================] - 1s 132us/step - loss: 0.4768
Epoch 4/75
7500/7500 [==============================] - 1s 129us/step - loss: 0.4675
Epoch 5/75
7500/7500 [==============================] - 1s 130us/step - loss: 0.4591
Epoch 6/75
7500/7500 [==============================] - 1s 130us/step - loss: 0.4483
Epoch 7/75
7500/7500 [==============================] - 1s 132us/step - loss: 0.4386
Epoch 8/75
7500/7500 [==============================] - 1s 134us/step - loss: 0.4271
Epoch 9/75
7500/7500 [==============================] - 1s 132us/step - loss: 0.4153
Epoch 10/75
7500/7500 [==============================] - 1s 133us/step - loss: 0.4059
Epoch 11/75
7500/7500 [==============================] - 1s 134us/step - loss: 0.3985
Epoch 12/75
7500/7500 [==============================] - 1s 135us/step - loss: 0.3904
Epoch 13/75
7500/7500 [==============================] - 1s 134us/step - loss: 0.3822
Epoch 14/75
7500/7500 [==============================] - 1s 137us/step -

...

/var/root/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.169169). Check your callbacks.
  % delta_t_median)


7500/7500 [==============================] - 1s 151us/step - loss: 0.2643
Epoch 44/75
7500/7500 [==============================] - 1s 151us/step - loss: 0.2645
Epoch 45/75
7500/7500 [==============================] - 1s 159us/step - loss: 0.2642
Epoch 46/75
7500/7500 [==============================] - 19s 3ms/step - loss: 0.2631
Epoch 47/75
7500/7500 [==============================] - 1s 151us/step - loss: 0.2642
Epoch 48/75
7500/7500 [==============================] - 1s 155us/step - loss: 0.2613
Epoch 49/75
7500/7500 [==============================] - 1s 156us/step - loss: 0.2634
Epoch 50/75
7500/7500 [==============================] - 1s 160us/step - loss: 0.2613
Epoch 51/75
1408/7500 [====>.........................] - ETA: 1s - loss: 0.2634

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/var/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-47a0c3a37558>", line 17, in <module>
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[redraw_callback])
  File "/var/root/anaconda3/lib/python3.6/site-packages/keras/models.py", line 963, in fit
    validation_steps=validation_steps)
  File "/var/root/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "/var/root/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1241, in _fit_loop
    callbacks.on_batch_end(batch_index, batch_logs)
  File "/var/root/anaconda3/lib/python3.6/site-packages/keras/callbacks.py", line 113, in on_batch_end
    callback.on_batch_end(batch, logs)
  File "/Users/alipeles/GoogleDrive/Jupyter Notebooks/LendingClub

KeyboardInterrupt: 